# FilePathing

In [ ]:
"""
# This is my code for navigating to my drive to get my images

from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/CS194-26/Proj1"
"""

# Imports

In [ ]:
%pylab inline

import os
from tqdm.notebook import tqdm
import cv2
import skimage as sk
from skimage import transform
import skimage.io as skio

!pip install line_profiler
%load_ext line_profiler

# Utils

Just some utils that help with image processing and formatting and io

naive_get_colors: returns the r, g, and b arrays by naivley cutting the source image (located at path) into thirds

im_from_colors: stacks the r, g, b channels into an image

show: just a small version of the existing show method that will increase the figure size automatically, allowing me to look at images in better detail

In [ ]:
"""
#Replace this code with some way to get the filepaths for all your images

image_paths = ['./Images/' + path for path in next(os.walk('./Images'))[2]]
big_paths = [path for path in image_paths if 'tif' in path]
small_paths = [path for path in image_paths if path not in big_paths]
"""


def naive_get_colors(path):
  im = imread(path)
  im = im.astype(float)
  height = im.shape[0]//3
  b = im[:height]
  g = im[height: 2*height]
  r = im[2*height: 3*height]
  return r, g, b

def im_from_colors(r, g, b):
  return np.dstack([r, g, b])

def show(im, figsize = 50, cmap=None):
  # Uses matplotlib in the backend to show images
  figure(figsize=(figsize,figsize))
  imshow(im, cmap=cmap)


Basic Image processing shit:


get_offset_maps: given 2 channels (one base and the other offset) return the overlaping parts of the two channels

overlaps_filter: given 3 images and all their offsets (x, y) from being all centered, this function returns the overlapping parts of the r, g, and b channels, so the resulting r, g, b overlaps can be stacked and shown as an image. This works whether the order of r, g, b is shifted or not... it will just return them in the same order that they were passed in

percent_crops: crops all the edges for r, g, and b by "percent" percent

In [ ]:
def get_offset_maps(base, offset_map, offset):
  """
  This is a helper function for exhaustive_search, but can be used more generally
  to return the part of each map that overlaps, given an offset

        base
     _______________
    |               |
    |        _______|_______
    |       |       |       |
    |_______|_______|       |
            |               |
            |_______________|

            offset map (all positive offsets for x, y in this case)
  """
  x, y = offset

  #Note: need to replace 0 in upper bound with None otherwise slicing returns []
  #lower bound: if positive, we want positive, if negative we want zero
  #upper bound: if positive, we want zero, if negative, we want negative
  base = base[max(0, y):min(0, y) if min(0, y) else None,
              max(0, x):min(0, x) if min(0, x) else None]

  #lower bound: if positive, we want zero, if negative, we want positive
  #upper bound: if positive, we want negative, if negative, we want zero
  offset_map = offset_map[max(0, -y):min(0, -y) if min(0, -y) else None,
                          max(0, -x):min(0, -x) if min(0, -x) else None]

  return base, offset_map


def overlaps_filter(r, g, b, r_offset, g_offset, b_offset):
  offsets = (r_offset, g_offset, b_offset)
  height, width = r.shape
  
  tops = [offset[1] for offset in offsets]
  lefts = [offset[0] for offset in offsets]
  bottoms = [offset[1] + height for offset in offsets]
  rights = [offset[0] + width for offset in offsets]

  top = max(tops)
  left = max(lefts)
  bottom = min(bottoms)
  right = min(rights)

  #here, instead of doing what we did with the offset maps, we just make all the
  # values positive so that we don't have to deal with the case of zero on the upper limit
  r = r[top - r_offset[1] : bottom - r_offset[1], left - r_offset[0] : right - r_offset[0] ]
  g = g[top - g_offset[1] : bottom - g_offset[1], left - g_offset[0] : right - g_offset[0] ]
  b = b[top - b_offset[1] : bottom - b_offset[1], left - b_offset[0] : right - b_offset[0] ]

  return r, g, b


def percent_crop(r, g, b, percent = .1):
  #crops by this percentage
  if r.shape != g.shape or g.shape != b.shape:
    raise Exception('edge_map_score: Color channels have unequal dimensions')

  i1 = int(r.shape[0] * percent)
  i2 = int(r.shape[1] * percent)

  r = r[i1:-i1, i2:-i2]
  g = g[i1:-i1, i2:-i2]
  b = b[i1:-i1, i2:-i2]

  return r, g, b


# Main Code

This is where all the meat is. Here we have all the allignment methods that actually do non-trivial stuff.

Allignment Scores:

**These need to be normalized for the number of pixels in the input channels**

Our objective should be to **maximize** these scores

In [ ]:
def l2_score(color1, color2):

  if color1.shape != color2.shape:
    raise Exception('edge_map_score: Color channels have unequal dimensions')

  color1_norm = np.sqrt(np.sum(color1*color1))
  color2_norm = np.sqrt(np.sum(color2*color2))

  score = np.sum(np.multiply(color1, color2))

  return score / (color1_norm * color2_norm)


def ssd(color1, color2):
  return -sum(sum((color1-color2)**2))/color1.size


exhaustive_search: this will use exhaustive search to find allignment. You will pass in one color channel as the base, and then set color1 and color2 to be the remaining two color channels. We will search for an allignment for both color 1 and color 2 within the ranges x_range_colori and y_range_colori for color i. It also takes in the score function that you want to use. We will use the convention that the score is trying to be maximized. 

full_exhaustive: takes in a path, your chosen assignments for base, color1, color2, and will plug it all into exhaustive_search, print the resulting image out along with the allignments (if verbose=True)

pyramid_search: this will use pyramid search to find allignment. You pass in your base color channel, the two other color channels, a searchrange for x, and y for both channels, a score function, a downsampling parameter that chooses how much you downsample by each time, an error_range that expands the normal window as we recurse back up by error_range amount, and a base case value to tell us at which window size we should stop getting more granular.

full_pyramid: another wrapper, same style as full_exhaustive



In [ ]:
def exhaustive_search(base, color1, color2, x_range_color1 = (-15, 15), y_range_color1 = (-15, 15),
                      x_range_color2 = (-15, 15), y_range_color2 = (-15, 15), score_fn = ssd):

  #make it so that color1, color2 and corresponding offsets point to the
  #underlying color slice and offset lists, which we will use
  #only the offset tuples should be modified. The colormaps should stay the same

  #do the exhaustive search itself, for color1
  best_score = -float('inf')
  best_offset = [0, 0]
  for x in range(*x_range_color1):
    for y in range(*y_range_color1):
      temp_base, temp_offset = get_offset_maps(base, color1, (x, y))
      score = score_fn(temp_base, temp_offset)
      if score > best_score:
        best_score = score
        best_offset = [x, y]

  color1_offset = best_offset

  #do the exhaustive search itself, for color2
  best_score = -float('inf')
  best_offset = [0, 0]
  for x in range(*x_range_color2):
    for y in range(*y_range_color2):
      temp_base, temp_offset = get_offset_maps(base, color2, (x, y))
      score = score_fn(temp_base, temp_offset)
      if score > best_score:
        best_score = score
        best_offset = [x, y]

  color2_offset = best_offset

  return color1_offset, color2_offset


def full_exhaustive(path, base_str = 'r', color1_str = 'g', color2_str = 'b', verbose = True):
  r, g, b = naive_get_colors(path)
  r = r/np.max(r)
  g = g/np.max(g)
  b = b/np.max(b)

  base = eval(base_str)
  color1 = eval(color1_str)
  color2 = eval(color2_str)

  color1_offset, color2_offset = exhaustive_search(*percent_crop(base, color1, color2, .3))

  r_offset = (0, 0) if 'r'==base_str else (color1_offset if color1_str == 'r' else color2_offset)
  g_offset = (0, 0) if 'g'==base_str else (color1_offset if color1_str == 'g' else color2_offset)
  b_offset = (0, 0) if 'b'==base_str else (color1_offset if color1_str == 'b' else color2_offset)

  if verbose:
    print('Color offsets:', 'Red: ', tuple(r_offset), 'Green: ', tuple(g_offset), 'Blue: ', tuple(b_offset))
  r, g, b = overlaps_filter(r, g, b, r_offset, g_offset, b_offset)
  show(im_from_colors(r, g, b))


def pyramid_search(base, color1, color2, searchrange = (-150, 150), score_fn = ssd, downsampling = 2, error_range = 0, base_case=10):
  #error range is the extra pixels we check on either side of the range to account for extra errors and wonkyness in the image
  #we will recurse when the range <= base_case

  #our base case
  if searchrange[1] - searchrange[0] <= base_case:
    color1_offset, color2_offset = exhaustive_search(base, color1, color2, searchrange, searchrange,
                                                     searchrange, searchrange, score_fn)
    return color1_offset, color2_offset
  
  #recursive case
  #get offsets for the downsampled image, modifying the range and the colors accordingly
  color1_offset, color2_offset = pyramid_search(base = sk.transform.rescale(base, 1/downsampling), 
                                                color1 = sk.transform.rescale(color1, 1/downsampling),
                                                color2 = sk.transform.rescale(color2, 1/downsampling), 
                                                searchrange = (searchrange[0]//downsampling, searchrange[1]//downsampling),
                                                score_fn = score_fn, downsampling = downsampling, 
                                                error_range = error_range)

  #upscale the offsets
  color1_offset = (color1_offset[0] * downsampling, color1_offset[1] * downsampling)
  color2_offset = (color2_offset[0] * downsampling, color2_offset[1] * downsampling)

  #get our ranges that the new offset could feasibly be in... plus the extra error_range
  x_range_color1 = (color1_offset[0] - (downsampling + error_range - 1), color1_offset[0] + downsampling + error_range)
  y_range_color1 = (color1_offset[1] - (downsampling + error_range - 1), color1_offset[1] + downsampling + error_range)
  x_range_color2 = (color2_offset[0] - (downsampling + error_range - 1), color2_offset[0] + downsampling + error_range)
  y_range_color2 = (color2_offset[1] - (downsampling + error_range - 1), color2_offset[1] + downsampling + error_range)
  
  return exhaustive_search(base, color1, color2, x_range_color1, y_range_color1,
                           x_range_color2, y_range_color2, score_fn)


def full_pyramid(path, base_str = 'r', color1_str = 'g', color2_str = 'b', verbose = True, searchrange=(-150, 150)):
  r, g, b = naive_get_colors(path)
  r = r/np.max(r)
  g = g/np.max(g)
  b = b/np.max(b)

  base = eval(base_str)
  color1 = eval(color1_str)
  color2 = eval(color2_str)

  color1_offset, color2_offset = pyramid_search(*percent_crop(base, color1, color2, .1), searchrange = searchrange)

  r_offset = (0, 0) if 'r'==base_str else (color1_offset if color1_str == 'r' else color2_offset)
  g_offset = (0, 0) if 'g'==base_str else (color1_offset if color1_str == 'g' else color2_offset)
  b_offset = (0, 0) if 'b'==base_str else (color1_offset if color1_str == 'b' else color2_offset)

  if verbose:
    print('Color offsets:', 'Red: ', tuple(r_offset), 'Green: ', tuple(g_offset), 'Blue: ', tuple(b_offset))
  r, g, b = overlaps_filter(r, g, b, r_offset, g_offset, b_offset)
  show(im_from_colors(r, g, b), figsize=20)


# Bells and Whistles

Image Preprocessing

edge_maps: this takes in r, g, b and comes up with a pretty comprehensive edge map by taking the absolute value of color gradients in the x, y, and xy directions (as opposed to the basic 'gradient' kernel with an 8 in the middle and -1 elsewhere). It also clips the gradients to a minimum and maximum number of standard deviations from the mean gradient. This function gives better r, g, b images to come up with offsets with, since color changes in the same places for all channels, even if the brightness of all areas isnt the same


In [ ]:
def edge_maps(r, g, b, clip_stds = (0, float('inf'))):
  #clip clips the edges at a certain std value and then rescales accordingly

  #r = cv2.GaussianBlur(r, (5,5), 0)
  #g = cv2.GaussianBlur(g, (5,5), 0)
  #b = cv2.GaussianBlur(b, (5,5), 0)

  x_edges = [cv2.Sobel(src=img, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) for img in (r, g, b)]
  y_edges = [cv2.Sobel(src=img, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) for img in (r, g, b)]
  xy_edges = [cv2.Sobel(src=img, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) for img in (r, g, b)]

  r = abs(x_edges[0]) + abs(y_edges[0]) + abs(xy_edges[0])
  g = abs(x_edges[1]) + abs(y_edges[1]) + abs(xy_edges[1])
  b = abs(x_edges[2]) + abs(y_edges[2]) + abs(xy_edges[2])

  r_std = np.std(r)
  r = np.clip(r, clip_stds[0] * r_std, clip_stds[1] * r_std)
  r = r/np.max(r)

  g_std = np.std(g)
  g = np.clip(g, clip_stds[0] * g_std, clip_stds[1] * g_std)
  g = g/np.max(g)

  b_std = np.std(b)
  b = np.clip(b, clip_stds[0] * b_std, clip_stds[1] * b_std)
  b = b/np.max(b)

  return r, g, b

def edge_pyramid(path, base_str = 'r', color1_str = 'g', color2_str = 'b', verbose = True, searchrange=(-150, 150)):
  r, g, b = naive_get_colors(path)
  r = r/np.max(r)
  g = g/np.max(g)
  b = b/np.max(b)

  base = eval(base_str)
  color1 = eval(color1_str)
  color2 = eval(color2_str)

  color1_offset, color2_offset = pyramid_search(*edge_maps(*percent_crop(base, color1, color2, .1), (0, 4)), searchrange = searchrange)

  r_offset = (0, 0) if 'r'==base_str else (color1_offset if color1_str == 'r' else color2_offset)
  g_offset = (0, 0) if 'g'==base_str else (color1_offset if color1_str == 'g' else color2_offset)
  b_offset = (0, 0) if 'b'==base_str else (color1_offset if color1_str == 'b' else color2_offset)

  if verbose:
    print('Color offsets:', 'Red: ', tuple(r_offset), 'Green: ', tuple(g_offset), 'Blue: ', tuple(b_offset))
  r, g, b = overlaps_filter(r, g, b, r_offset, g_offset, b_offset)
  show(im_from_colors(r, g, b), figsize=20)



# Main

In [ ]:
# It is assumed that at this point, you have a 
#   list 'image_paths' of all image paths

len(image_paths)


In [ ]:
i=0
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=1
print(image_paths[i])
full_pyramid(image_paths[i], 'g', 'r', 'b')


In [ ]:
i=2
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=3
print(image_paths[i])
full_pyramid(image_paths[i], 'r', 'g', 'b', searchrange=(-15, 15))


In [ ]:
i=4
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=5
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g', searchrange = (-15, 15))


In [ ]:
i=6
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g', searchrange=(-15, 15))


In [ ]:
i=7
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=8
print(image_paths[i])
full_pyramid(image_paths[i], 'r', 'g', 'b')


In [ ]:
i=9
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=10
print(image_paths[i])
full_pyramid(image_paths[i], 'r', 'g', 'b')


In [ ]:
i=11
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=12
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


In [ ]:
i=13
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


Other Images in the Collection

In [ ]:
full_pyramid('./Images/other1.tif', 'g', 'r', 'b')


In [ ]:
full_pyramid('./Images/other2.tif', 'g', 'r', 'b')


In [ ]:
full_pyramid('./Images/other3.tif', 'g', 'r', 'b')


In [ ]:
full_pyramid('./Images/other4.tif', 'g', 'r', 'b')


In [ ]:
full_pyramid('./Images/other5.tif', 'g', 'r', 'b')


# Bells and Whistles Demo

Here we can see the regular algorithm failing on the emir. The first photo has yellow smearing on the sides of the man. The second is even worse. These are bad selections of base color since the blue is so saturated.

In [ ]:
i=1
print(image_paths[i])
full_pyramid(image_paths[i], 'r', 'g', 'b')


In [ ]:
i=1
print(image_paths[i])
full_pyramid(image_paths[i], 'b', 'r', 'g')


This image uses edge maps to better allign the image

In [ ]:
i=1
print(image_paths[i])
edge_pyramid(image_paths[i], 'b', 'r', 'g')